# Тестирование сервиса в ручную

In [11]:
from pathlib import Path

DATA_PATH = Path('../../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

LOG_DATA = Path('../../logs/')
LOG_DATA.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [12]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [13]:
from app.settings import settings
from app.models.openai_service import OpenAIService

# Проверим API ключ
print(f"OpenAI API Key загружен: {'да' if settings.OPENAI_API_KEY else 'нет'}")


OpenAI API Key загружен: да


In [14]:
llm = OpenAIService(api_key=settings.OPENAI_API_KEY, model_name="gpt-4.1-nano")


In [15]:
messages = [{"role": "user", "content": "Привет, как тебя зовут?"}]
response = llm.chat(messages=messages, stream=False)

print("Ответ модели:")
print(response)


2025-08-14 15:56:12 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Ответ модели:
Привет! Я — искусственный интеллект, и у меня нет имени, но ты можешь обращаться ко мне как угодно. Как я могу помочь тебе сегодня?


In [16]:
messages = [{"role": "user", "content": "Напиши короткий рассказ про кота"}]

print("Потоковый ответ:")
for chunk in llm.chat(messages=messages, stream=True):
    print(chunk, end="", flush=True)


Потоковый ответ:
2025-08-14 15:56:12 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Жил-был в одном маленьком доме серый кот по имени Тимка. Он был любопытным и добрым, всегда готовым к приключениям. Каждое утро Тимка просыпался с первыми лучами солнца и бегал по саду в поисках ярких бабочек. Однажды, пробежав мимо цветущих кустов, он наткнулся на потерявшуюся птичку. Не раздумывая, кот аккуратно поднял её лапой и аккуратно отнес на безопасное место. С тех пор Тимка стал не только любимцем хозяев, но и другом для всех животных вокруг. И каждый вечер, уютно свернувшись у камина, он чувствовал тепло и радость от того, что помогает окружающему миру.

In [17]:
from app.services.token_manager import TokenManager

tm = TokenManager("gpt-4.1-nano")
text = "Привет, мир!"
print(f"Текст: {text}")
print(f"Токенов: {tm.count_tokens(text)}")


Текст: Привет, мир!
Токенов: 5
